## Notebook to retrieve IWP from GMI TB and other auxiliary data using QRNN, uses quantNN

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4

from quantnn.metrics import ScatterPlot
from quantnn.models.pytorch.logging import TensorBoardLogger
from quantnn.transformations import LogLinear, Log, Log10, Id

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData import gmiData
import os
import time
from quantnn.models.pytorch.fully_connected import FullyConnected
from torch import optim
from quantnn.qrnn import QRNN
from torch.optim import Adam

import time
#time.sleep(60 * 60 * 1.5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Inputs

In [15]:
quantiles         = np.linspace(0.01, 0.99, 51)

#quantiles         =  np.array([0.2, 3 , 16 ,50 ,84 ,97, 99.8])

batchSize         = 256


n_layers          = 5
n_neurons         = 256


#inputs            = np.array(["ta", "t2m",  "wvp", "z0", "lat", "stype"])
inputs            = np.array(["ta", "t2m",  "wvp", "z0", "stype"])
ninputs           = len(inputs) + 2 + 7

outputs           = "iwp"

latlims           = [0, 65]


#outputfile        = os.path.join(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/"), 
#                                                    filename)

In [16]:
np.linspace(0.01, 0.99, 51)

array([0.01  , 0.0296, 0.0492, 0.0688, 0.0884, 0.108 , 0.1276, 0.1472,
       0.1668, 0.1864, 0.206 , 0.2256, 0.2452, 0.2648, 0.2844, 0.304 ,
       0.3236, 0.3432, 0.3628, 0.3824, 0.402 , 0.4216, 0.4412, 0.4608,
       0.4804, 0.5   , 0.5196, 0.5392, 0.5588, 0.5784, 0.598 , 0.6176,
       0.6372, 0.6568, 0.6764, 0.696 , 0.7156, 0.7352, 0.7548, 0.7744,
       0.794 , 0.8136, 0.8332, 0.8528, 0.8724, 0.892 , 0.9116, 0.9312,
       0.9508, 0.9704, 0.99  ])

## read in training data

In [17]:
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_jan_esa.nc"), 
               inputs, outputs, latlims = latlims, pratio = None,
               batch_size = batchSize, transform = None)  

norm = data.norm

validation_data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_test_jan_esa.nc"), 
               inputs, outputs, latlims = latlims, normalise = norm, pratio = None,
               batch_size = batchSize, transform = None)  


training_data      = DataLoader(data, batch_size=None, num_workers=1, pin_memory=True)
validation_data    = DataLoader(validation_data, batch_size=None, num_workers=1, pin_memory=True)

{0: (268.5665796717347, 20.29343380365656), 1: (264.9757803213727, 23.312992560374052), 2: (265.45348009405046, 13.75845443481109), 3: (257.0582191483074, 9.455841612721185), 4: (286.3129319424382, 14.495873546238812), 5: (22.564311828462827, 16.375402352734177), 6: (164.17300322756427, 452.12711641524135)}
{0: (268.5665796717347, 20.29343380365656), 1: (264.9757803213727, 23.312992560374052), 2: (265.45348009405046, 13.75845443481109), 3: (257.0582191483074, 9.455841612721185), 4: (286.3129319424382, 14.495873546238812), 5: (22.564311828462827, 16.375402352734177), 6: (164.17300322756427, 452.12711641524135)}


In [18]:
norm.stats


{0: (268.5665796717347, 20.29343380365656),
 1: (264.9757803213727, 23.312992560374052),
 2: (265.45348009405046, 13.75845443481109),
 3: (257.0582191483074, 9.455841612721185),
 4: (286.3129319424382, 14.495873546238812),
 5: (22.564311828462827, 16.375402352734177),
 6: (164.17300322756427, 452.12711641524135)}

In [19]:
data.x.shape

(741745, 15)

In [20]:
validation_data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_test_jan_lpa_pr1.nc"), 
               inputs, outputs, latlims = latlims, normalise = norm, pratio = None,
               batch_size = batchSize, transform = None) 
validation_data.x.shape

{0: (268.5665796717347, 20.29343380365656), 1: (264.9757803213727, 23.312992560374052), 2: (265.45348009405046, 13.75845443481109), 3: (257.0582191483074, 9.455841612721185), 4: (286.3129319424382, 14.495873546238812), 5: (22.564311828462827, 16.375402352734177), 6: (164.17300322756427, 452.12711641524135)}


(247248, 15)

In [21]:
for i in ["0"]:
    
    filename          = "qrnn_gmi_nn_lpaesa_pr1_v" + i + "_jan_v.nc"
    outputfile        = os.path.join(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/"), 
                                                        filename)

    print(filename)
    skip_connections  = True
    batch_norm        = True

    skip_connections  = True
    batch_norm        = True
    transform         = LogLinear()

    model             = FullyConnected(ninputs,
                           quantiles.size,
                           n_layers,
                           n_neurons,
                           skip_connections=skip_connections,
                           batch_norm=batch_norm)

    #model = FullyConnected(n_inputs=ninputs, n_outputs=len(quantiles), n_layers=5, width=256)

    qrnn             = QRNN(quantiles, ninputs,  model, transformation = transform)

    metrics          = ["MeanSquaredError", "Bias", "CRPS", "CalibrationPlot"]

    n_epochs         = 200
    logger = TensorBoardLogger(n_epochs)

    scatter_plot    = ScatterPlot(bins=np.logspace(-2, 2, 21), log_scale=True)
    metrics.append(scatter_plot)

    for lr, n_epochs in zip([1e-2, 1e-3, 1e-4], [50, 50, 100]):

    #for lr, n_epochs in zip([5e-3, 1e-4, 1e-5], [20, 50, 100]):

    #for lr, n_epochs in zip([0.01], [120]):

        #optimizer = Adam(qrnn.model.parameters(), lr=lr)

        optimizer = optim.SGD(model.parameters(), lr = lr, momentum = 0.9)

        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

        qrnn.train(training_data=training_data,
                   validation_data=validation_data,
                   n_epochs=n_epochs,
                   mask=-999,
                   device="cuda",
                   logger=logger,
                   metrics=metrics,
                   optimizer=optimizer, 
                   scheduler=scheduler,
                  );

    qrnn.save(outputfile)

qrnn_gmi_nn_lpaesa_pr1_v0_jan_v.nc


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
qrnn.save(outputfile)

In [ ]:
%matplotlib inline

y_pre = []
y = []
y_prior = []
y_pos_mean = []
x_in = []

nbatch = len(validation_data)
print (nbatch)
for i in range(nbatch):
    
    xx, yy = validation_data[i]
    
    x = xx.detach().numpy() 

    y_pre.append(qrnn.predict(xx)) 
    y_pos_mean.append((qrnn.posterior_mean(xx)))
       
    y.append(yy.detach().numpy())
    x_in.append(x)

x_in = np.concatenate(x_in, axis = 0)
y_pre = np.concatenate(y_pre, axis = 0)
y = np.concatenate(y, axis= 0)
y_pos_mean = np.concatenate(y_pos_mean, axis = 0)

plt.rcParams.update({'font.size': 20})
bins1 = np.arange(0, 30, 0.1)



In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
bins1 = np.arange(0, 30, 0.1)

ax.hist(y_pre[:, imedian], bins1, density = True , histtype = "step", label = "predicted")


ax.hist(y, bins1, density = True, histtype = "step", label = "observed")
ax.set_yscale('log')
ax.set_xscale('log')

ax.legend()
ax.set_ylabel("PDF")
ax.set_xlabel("IWP[kg/m2]")

In [ ]:
import scipy
from matplotlib import ticker, cm
xyrange = [[0, 25], [0, 25]] # data range


bins = [50, 50] # number of bins
hh, locx, locy = np.histogram2d(y, y_pos_mean, 
                                range=xyrange, bins=bins, density = True)
posx = np.digitize(y, locx)
posy = np.digitize(y_pos_mean, locy)

fig, ax = plt.subplots(1, 1, figsize = [10, 8])
cs = ax.contourf(np.flipud(hh.T),
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
cbar = fig.colorbar(cs)
ax.set_ylim([0, 20])
ax.set_xlim([0, 20])
xy = np.arange(0, 13, 1)
yy = xy
ax.plot(xy, yy)
ax.set_ylabel("IWP Predicted [kg/m2]")
ax.set_xlabel("IWP Observed [kg/m2]")
ax.grid('on')
fig.savefig("qrnn.png", bbox_inches = "tight")
#ax.set_yscale('log')
#ax.set_xscale('log')